In [28]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

def solve():
    input = sys.stdin.readline

    T = int(input())
    for _ in range(T):

        K = int(input())
        files = list(map(int, input().split()))

        S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
        for i in range(1, K + 1):
            S[i] = S[i-1] + files[i - 1]

        # dp[i][j] i부터 j 까지 결합하는데 최소 시간
        dp = [[0] * (K + 1) for _ in range(K + 1)]

        for length in range(2, K + 1):
            for i in range(1, K - length + 2):
                j = i + length - 1
                candi = [dp[i][k] + dp[k+1][j] for k in range(i, j)]

                # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
                dp[i][j] = min(candi) + (S[j] - S[i-1])

        print(dp[1][K])
solve()

300
864


In [35]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

input = sys.stdin.readline

def solve():
    K = int(input())
    files = list(map(int, input().split()))

    S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
    for i in range(1, K + 1):
        S[i] = S[i-1] + files[i - 1]

    # dp[i][j] i부터 j 까지 결합하는데 최소 시간
    dp = [[0] * (K + 1) for _ in range(K + 1)]
    min_k = [[0] * (K + 1) for _ in range(K + 1)]

    for i in range(1, K):
        dp[i][i+1] = files[i-1] + files[i]
        min_k[i][i+1] = i # 분기점 저장

    for length in range(3, K + 1):
        for i in range(1, K - length + 2):
            j = i + length - 1

            start_k = min_k[i][j-1]
            end_k = min_k[i+1][j]

            min_val = float('inf')
            opt_k = -1

            # 크누스 최적화 적용 (증명됨) 
            for k in range(start_k, end_k + 1):
                val = dp[i][k] + dp[k+1][j]
                if val < min_val:
                    min_val = val
                    opt_k = k
            
            # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
            dp[i][j] = min_val + (S[j] - S[i-1])
            min_k[i][j] = opt_k

    print(dp[1][K])

T = int(input())
for _ in range(T):
    solve()

300
864


In [ ]:
# 2	11049	 행렬 곱셈 순서
import sys

from io import StringIO
sys.stdin = StringIO('''3
5 3
3 2
2 6''')

input = sys.stdin.readline

def solve():

    N = int(input())
    mats = [(0, 0)]
    for _ in range(N):
        mats.append(tuple(map(int, input().split())))

    dp = [[0] * (N + 1) for _ in range(N + 1)]

    for length in range(2, N + 1):
        for start in range(1, N - length + 2):
            end = start + length - 1

            row_start = mats[start][0]
            col_end = mats[end][1]
            min_multi_val = float('inf')

            dp_start = dp[start]
            for mid in range(start, end):
                lv = dp_start[mid]
                rv = dp[mid + 1][end]
                new = row_start * mats[mid][1] * col_end
                candi_val = lv + rv + new

                if candi_val < min_multi_val:
                    min_multi_val = candi_val

            dp[start][end] = min_multi_val

    print(dp[1][N])


solve()

90


In [53]:
# 3	1520	 내리막 길
import sys
sys.setrecursionlimit(10**9)

from io import StringIO
sys.stdin = StringIO('''4 5
50 45 37 32 30
35 50 40 20 25
30 30 25 17 28
27 24 22 15 10''')

input = sys.stdin.readline

def solve():
    M, N = map(int, input().split())
    mountain = [list(map(int, input().split())) for _ in range(M)]

    directions = [
        (+1, 0), (0, +1),
        (-1, 0), (0, -1)
    ]

    # dp[i][j] = ij 에서 M, N 까지 도달 할 수 있는 경우의 수 

    dp = [[-1] * N for _ in range(M)]

    def dfs(r, c):

        if r == M-1 and c == N-1:
            return 1
        
        if dp[r][c] != -1:
            return dp[r][c]
        
        ways = 0
        for dr, dc in directions:
            nr, nc = r + dr, c + dc

            if 0 <= nr < M and 0 <= nc < N:
                if mountain[r][c] > mountain[nr][nc]:
                    ways += dfs(nr, nc)
        
        dp[r][c] = ways
        return ways
        
    print(dfs(0, 0))
solve()


3


In [58]:
# 3	1520	 내리막 길
import sys
from heapq import heappop, heappush

from io import StringIO
sys.stdin = StringIO('''4 5
50 45 37 32 30
35 50 40 20 25
30 30 25 17 28
27 24 22 15 10''')

input = sys.stdin.readline


def solve():
    M, N = map(int, input().split())
    mountain = [list(map(int, input().split())) for _ in range(M)]

    directions = [
        (+1, 0), (0, +1),
        (-1, 0), (0, -1)
    ]

    # dp[r][c] : (0, 0) 출발 (r, c) 까지 도달하는 경로 수
    dp = [[0] * N for _ in range(M)]
    dp[0][0] = 1

    pq = [(-mountain[0][0], 0, 0)]  # (-마이너스 높이, row, col)

    while pq:
        h, r, c = heappop(pq)
        height = -h

        if r == M-1 and c == N-1:
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc

            if 0 <= nr < M and 0 <= nc < N:
                nxt_h = mountain[nr][nc]
                if height > nxt_h:
                    if dp[nr][nc] == 0:
                        heappush(pq, (-nxt_h, nr, nc))
                    dp[nr][nc] += dp[r][c]
    print(dp[M-1][N-1])
solve()

3


In [ ]:
# 4	2629	 양팔저울
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 5	2293	 동전 1
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 6	7579	 앱
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()

In [ ]:
# 7	11062	 카드 게임
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline

def solve():

    pass

solve()